#### TO RUN SPARK CLUSTER OUTSIDE THIS NOTEBOOK
- insructions can be followed from this docs [https://spark.apache.org/docs/3.5.3/spark-standalone.html]
- got the the directory where spark is installed, where the (echo $SPARK_HOME) is located, then run this bash script: ./sbin/start-master.sh
- it will be on port:8080 not port:4040
- get the URL, replace the (local[*])
- TO ADD EXCUTORS run: ./sbin/start-worker.sh <master-spark-URL>  # change worker to slave in some versions
- to the stop the spark cluster
    run:./sbin/stop-master.sh ...to stop the master node
    run:./sbin/stop-worker.sh ...to stop the excutor or slave

In [4]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark_url ="spark://mle-zm-spark.us-west1-a.c.resonant-time-434823-n0.internal:7077"
spark = SparkSession.builder \
    .master(spark_url) \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/12/05 08:13:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [17]:
spark

In [6]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [7]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [8]:
df_green = df_green\
    .withColumnRenamed('lpep_pickup_datetime','pickup_datetime')\
    .withColumnRenamed('lpep_dropoff_datetime','dropoff_datetime')

In [9]:
df_yellow = df_yellow\
    .withColumnRenamed('tpep_pickup_datetime','pickup_datetime')\
    .withColumnRenamed('tpep_dropoff_datetime','dropoff_datetime')

In [10]:
common_columns = []
yellow_columns =  set(df_yellow.columns)

for col in df_green.columns:
    if col in yellow_columns:
        common_columns.append(col)

In [11]:
common_columns = [
    'VendorID',
    'pickup_datetime',
    'dropoff_datetime',
    'store_and_fwd_flag',
    'RatecodeID',
    'PULocationID',
    'DOLocationID',
    'passenger_count',
    'trip_distance',
    'fare_amount',
    'extra',
    'mta_tax',
    'tip_amount',
    'tolls_amount',
    'improvement_surcharge',
    'total_amount',
    'payment_type',
    'congestion_surcharge']

In [12]:
df_green_sel = df_green\
    .select(common_columns)\
    .withColumn('service_type', F.lit('green'))

In [13]:
df_yellow_sel = df_yellow\
    .select(common_columns)\
    .withColumn('service_type', F.lit('yellow'))

In [14]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [15]:
df_trips_data.registerTempTable('trips_data')

/home/MERHAWI/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [16]:
df_result = spark.sql(""" 
SELECT   
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc("month", "pickup_datetime")  AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_monthly_passenger_count,
    AVG(trip_distance) AS avg_monthly_trip_distance

FROM 
    trips_data
GROUP BY 
    1, 2, 3
""")

In [ ]:
df_result.write.parquet('data/report/revenue', mode='overwrite')